# Dreambooth Using Stable Diffusion
Notebook implementation by Abhishek Sharma
Latest information on: https://github.com/abhisheksh1304/Dreambooth-Using-Stable-Diffusion.git

## Build Environment

In [ ]:
!git clone https://github.com/abhisheksh1304/Dreambooth-Using-Stable-Diffusion.git


In [ ]:
!pip install numpy==1.23.1
!pip install pytorch-lightning==1.7.6
!pip install csv-logger
!pip install torchmetrics==0.11.1
!pip install torch-fidelity==0.3.0
!pip install albumentations==1.1.0
!pip install opencv-python==4.7.0.72
!pip install pudb==2019.2
!pip install omegaconf==2.1.1
!pip install pillow==9.4.0
!pip install einops==0.4.1
!pip install transformers==4.25.1
!pip install kornia==0.6.7
!pip install diffusers[training]==0.3.0
!pip install captionizer==1.0.1
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -e .
!pip install huggingface_hub
!pip install gitpython

# Download the 1.5 model from hugging face


You can also provide your own v1.5 model for training by uploading it and renaming it to "model.ckpt".  It should be in the same directory as in which this notebook is present.


In [ ]:
# Download the 1.5 sd model
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

# Regularization Images (Skip this section if you are uploading your own or using the provided images)

Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.


In [1]:
dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# remove temp folder now it is empty. 
!rm -rf Stable-Diffusion-Regularization-Images-{dataset}

clear_output()
print("✅ \033[92mRegularization Images downloaded.\033[0m")

## Dreambooth Training Environment Setup

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
#@title 7. Final Setup & Training

#@markdown This isn't used for training, just used to name the folders etc
project_name = "Dreambooth_Project" #@param {type:"string"}

#@markdown This is the unique token i.e. you can use a nonsensical word like zwx or your name.
token = "abhishek" #@param {type:"string"}

#@markdown Match class_word to the category of the regularization images you chose above.
class_word = "person" #@param ["man", "person", "woman"] {allow-input: true}

# MAX STEPS
#@markdown How many steps do you want to train for?
max_training_steps = 2000 #@param {type:"integer"}

#@markdown If you are training a person's face, set this to True
i_am_training_a_persons_face = True #@param {type:"boolean"}
flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5

#@markdown Would you like to save a model every X steps? (Example: 250 would output a trained model at 250, 500, 750 steps, etc)
save_every_x_steps = 500 #@param {type:"integer"}


reg_data_root = "./regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --project_name "{project_name}" \
 --debug False \
 --max_training_steps {max_training_steps} \
 --token "{token}" \
 --training_model "model.ckpt" \
 --training_images "./training_images" \
 --regularization_images "{reg_data_root}" \
 --class_word "{class_word}" \
 --flip_p {flip_p_arg} \
 --save_every_x_steps {save_every_x_steps}

After Training, Model chekpoints can be found in trained_models folder

# Big Important Note!

The way to use your token is `<token> <class>` ie `abhishek person` and not just `abhishek`

## Generate Images With Your Trained Model!

In [ ]:

!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 30 \
 --ckpt "./trained_models/2024-07-15T23-32-06_project_dreambooth_09000_steps_32_training_images_abhi_token_token_person_class_word.ckpt" \
 --prompt "abhishek person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"
